# Модуль ```functools```

Модуль ```functools``` содержит большое количество декораторов и функций для разных задачь. Туда включены: 
- декораторы для кеширования результатов функций (```cache```, ```lru_cache``` и др.);
- декораторы для облегчения написания декораторов и классов(```wraps```, ```total_ordering```);
- декораторы для сокращения количества передаваемых аргументов (```partial```);
- и другие.

Рассмотрим некоторые из них.

Декоратор ```lru_cache(maxsize=128, typed=False)``` предназначен для [LRU кеширования](https://en.wikipedia.org/wiki/Cache_replacement_policies#Least_recently_used_(LRU)). Это означает, что он "запоминает" результаты и аргументы вызова функций, и, когда функция повторно вызывается с такими же аргументами, то результат не вычисляется, а возвращается сохраненное значение. Это очень удобно, если функция выполняет тяжеловесные вычисления. 

```lru_cache``` использует в качестве кэша словарь, поэтому все аргументы должны быть хешируемыми.

Аргумент ```maxsize``` определяет размер памяти, ```lru_cache``` удаляет из кеша слишком старые результаты, тем самым экономя память. Если ```maxsize``` установлен в ```None```, кэш может возрастать бесконечно. Также функция наиболее эффективна, если ```maxsize``` это степень двойки.

Декоратор ```lru_cache``` может учитывать типы аргументов, для этого используется аргумент ```typed```. Если он установлен в ```True```, аргументы функции с разными типами будут кэшироваться отдельно. Например, ```f(3)``` и ```f(3.0)``` будут считаться разными вызовами, возвращающие, возможно, различный результат.

Чтобы помочь измерить эффективность кэширования и отрегулировать размер кэша, обёрнутая функция дополняется функцией ```cache_info()```, возвращающая ```namedtuple```, показывающий попадания в кэш, промахи, максимальный размер и текущий размер. В многопоточном окружении, количество попаданий и промахов будет приблизительным.

Также имеется функция ```cache_clear()``` для очистки кэша.

Оригинальная функция доступна через атрибут ```__wrapped__```.

In [4]:
from functools import lru_cache

@lru_cache(maxsize=5)
def factorial(n):
    if n == 0:
        return 1
    return n * factorial(n - 1)


print(f'{factorial(5) = }')
print(f'{factorial(10) = }')
print(f'{factorial(100) = }')
print(f'{factorial(6) = }')
print(f'{factorial(100) = }')
print(f'{factorial(5) = }')
print('-' * 50)

print(f'{factorial.cache_info()}')

factorial(5) = 120
factorial(10) = 3628800
factorial(100) = 93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000
factorial(6) = 720
factorial(100) = 93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000
factorial(5) = 120
--------------------------------------------------
CacheInfo(hits=3, misses=208, maxsize=5, currsize=5)


Декоратор ```partial(func, *args, **keywords)``` используется для частичного определения аргументов функции до ее непосредственного вызова. В момент вызова декорируемой функции ```func```, помимо явно передаваемых аргументов, будут переданы аргументы из ```args``` и ```keywords``` декоратора ```partial```.

In [16]:
from functools import partial

min_5 = partial(min, 5)
print(f'{min_5 = }')
print(f'{type(min_5) = }')

print(f'{min_5(10, 2) = }')
print(f'{min_5(10) = }')

min_5 = functools.partial(<built-in function min>, 5)
type(min_5) = <class 'functools.partial'>
min_5(10, 2) = 2
min_5(10) = 5


Это будет эквивалентно

In [ ]:
sum_5 = lambda *args: min(5, *args)

В других языках программирования можно создать несколько функций с одним названием, но с разными типами аргументов.

Как проделать тоже самое только в Python? Конечно, вы можете вручную проверять типы аргументов, но это не всегда бывает удобно. В модуле ```functools``` есть декоратор ```singledispatch```, который позволяет реализовать этот функционал.

Реализуем аналог функции ```range``` в нескольких вариантах исполнения: для целых чисел и для чисел с плавающей точкой.

Декоратором ```singledispatch``` декорируется базовая функция, она будет использоваться, если никакие остальные реализации не подходят.

С помощью метода ```register``` отмечаются остальные реализации. Он может принимать тип, для которого эта реализация будет использоваться. 

У этого подхода есть один недостаток. Предполагается, что у всех позиционных аргументов тип одинаковый.

Подробное описание и большое количество примеров см. в [документации](https://docs.python.org/3/library/functools.html).

In [30]:
from functools import singledispatch

@singledispatch
def my_range(start, stop, step):
    print('base func')

@my_range.register(int)
def _(start, stop, step):
    for i in range(start, stop, step):
        print(i)

@my_range.register(float)
def _(start, stop, step):
    while start < stop:
        print(start)
        start += step


my_range(1, 3, 1)
print('-' * 25)
my_range(1., 3., 0.5)
print('-' * 25)
my_range('a', 'b', 'c')

1
2
-------------------------
1.0
1.5
2.0
2.5
-------------------------
base func


Функция ```update_wrapper``` и ее аналог декоратор ```wraps``` нужны для копирования служебной информации присоздании собственных декораторов: имени функции, документации и другого.

In [1]:
from functools import wraps

def foo(f):
    @wraps(f)
    def inner():
        print(f'{f.__name__}')
        f()
    return inner

@foo
def bar():
    """Бесполезная функция"""
    print('(－‸ლ)')

print(f'{bar.__name__}')
print(f'{bar.__doc__}')

bar
Бесполезная функция


Декоратор ```total_ordering``` очень полезен при работе с классами. Он позволяет автоматически реализовывать все виды операторов сравнения. Для этого достаточно, чтобы класс реализовал только два метода: метод сравнения на равенство ```__eq__``` и один из методов ```__lt__```, ```__le__```, ```__gt__```, или ```__ge__```. Это существенно упрощает работу.

# Полезные ссылки

- [Документация](https://docs.python.org/3/library/functools.html)
- [PEP 443 -- Single-dispatch generic functions](https://www.python.org/dev/peps/pep-0443/)